<a href="https://colab.research.google.com/github/devyanighildiyal/Calender-Management-System/blob/main/google_girl_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"devyanighildiyal","key":"79e66e32ca432d1453ce907ad728606e"}'}

In [ ]:
import os
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Secure the API key

In [ ]:
!kaggle datasets download -d snehaanbhawal/resume-dataset

Dataset URL: https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset
License(s): CC0-1.0
100% 62.5M/62.5M [00:00<00:00, 173MB/s]
100% 62.5M/62.5M [00:00<00:00, 154MB/s]


In [ ]:
!unzip -q resume-dataset.zip -d dataset/

In [ ]:
!pip install pymupdf
!pip install pdfplumber
!pip install pytesseract
!sudo apt install tesseract-ocr
import os
import fitz  # PyMuPDF
import pdfplumber
import pytesseract
from PIL import Image
import pandas as pd
import re

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 22 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,990 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF using PyMuPDF (fastest method)."""
    try:
        with fitz.open(pdf_path) as doc:
            text = ""
            for page in doc:
                text += page.get_text("text") + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
        return None

In [ ]:
resume_folder = "/content/dataset/data/data"  # Main folder

resumes_data = []

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF using PyMuPDF (fast method)."""
    try:
        with fitz.open(pdf_path) as doc:
            text = ""
            for page in doc:
                text += page.get_text("text") + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
        return None

# Walk through all subdirectories and find PDFs
for root, _, files in os.walk(resume_folder):
    category = os.path.basename(root)  # Get subfolder name as occupation
    for file in files:
        if file.endswith(".pdf"):
            pdf_path = os.path.join(root, file)
            text = extract_text_from_pdf(pdf_path)
            if text:
                resumes_data.append({"filename": file, "category": category, "resume_text": text})

# Convert to DataFrame
resume_df = pd.DataFrame(resumes_data)

# Show some results
print(resume_df.head())


       filename       category  \
0  81508860.pdf  DIGITAL-MEDIA   
1  58915642.pdf  DIGITAL-MEDIA   
2  17132168.pdf  DIGITAL-MEDIA   
3  19444529.pdf  DIGITAL-MEDIA   
4  21125113.pdf  DIGITAL-MEDIA   

                                         resume_text  
0  DIGITAL MARKETING DIRECTOR\nSummary\nWith a ba...  
1  DIGITAL M&E RESEARCH INTERN\nSkills\nMicrosoft...  
2  SOCIAL ENGAGE SALES\nSummary\nMy Current role ...  
3  VIDEOGRAPHER\nProfessional Profile\nCreative i...  
4  MEDIA SUPPORT SPECIALIST\nProfessional Summary...  


In [ ]:
def clean_resume(text):
    """Preprocess resume text: remove special characters & extra spaces."""
    text = re.sub(r'\n+', ' ', text)  # Remove excessive new lines
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    return text.lower()

resume_df["cleaned_text"] = resume_df["resume_text"].apply(clean_resume)

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

def clean_resume(text):
    """Cleans resume text by removing unwanted characters."""
    text = re.sub(r'\n+', ' ', text)  # Remove excessive new lines
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

resume_df["cleaned_text"] = resume_df["resume_text"].apply(clean_resume)

# Show cleaned data
print(resume_df.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


       filename       category  \
0  81508860.pdf  DIGITAL-MEDIA   
1  58915642.pdf  DIGITAL-MEDIA   
2  17132168.pdf  DIGITAL-MEDIA   
3  19444529.pdf  DIGITAL-MEDIA   
4  21125113.pdf  DIGITAL-MEDIA   

                                         resume_text  \
0  DIGITAL MARKETING DIRECTOR\nSummary\nWith a ba...   
1  DIGITAL M&E RESEARCH INTERN\nSkills\nMicrosoft...   
2  SOCIAL ENGAGE SALES\nSummary\nMy Current role ...   
3  VIDEOGRAPHER\nProfessional Profile\nCreative i...   
4  MEDIA SUPPORT SPECIALIST\nProfessional Summary...   

                                        cleaned_text  
0  digital marketing director summary background ...  
1  digital research intern skills microsoft offic...  
2  social engage sales summary current role repre...  
3  videographer professional profile creative ind...  
4  media support specialist professional summary ...  


In [ ]:
import spacy

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Predefined list of skills (expand this list as needed)
skill_list = [
    "python", "java", "sql", "machine learning", "deep learning", "data science",
    "nlp", "excel", "tableau", "pandas", "keras", "tensorflow", "scikit-learn"
]

def extract_skills(text):
    """Extracts skills from resume using NLP matching."""
    doc = nlp(text)
    extracted_skills = set()

    for token in doc:
        word = token.text.lower()
        if word in skill_list:
            extracted_skills.add(word)

    return ", ".join(extracted_skills)

resume_df["extracted_skills"] = resume_df["cleaned_text"].apply(extract_skills)

# Show extracted skills
print(resume_df[["filename", "extracted_skills"]].head())

       filename extracted_skills
0  81508860.pdf                 
1  58915642.pdf      java, excel
2  17132168.pdf                 
3  19444529.pdf                 
4  21125113.pdf            excel


In [ ]:
def extract_experience(text):
    """Extracts years of experience from resume text."""
    experience = re.findall(r'(\d+)\s+years?', text)  # Find patterns like '3 years'
    if experience:
        return max(map(int, experience))  # Return the highest number found
    return 0  # Default to 0 if no experience found

resume_df["years_experience"] = resume_df["cleaned_text"].apply(extract_experience)

# Show extracted experience
print(resume_df[["filename", "years_experience"]].head())


       filename  years_experience
0  81508860.pdf                 0
1  58915642.pdf                 0
2  17132168.pdf                 0
3  19444529.pdf                 0
4  21125113.pdf                 4


In [ ]:
resume_df.to_csv("/content/processed_resumes.csv", index=False)
print("Processed resume data saved in Colab!")

Processed resume data saved in Colab!


In [ ]:
from google.colab import files

files.download("/content/processed_resumes.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>